# [실습05] 합성공 신경망(2)_Transfer Learning
## 휴먼지능정보공학과 201910841 최다경

## https://github.com/DaGyeongChoi

## 1.Settings
### 1) Important required libraries

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')
#!cd /content/gdrive/MyDrive/
#%cd /content/gdrive/MyDrive/ex_05/

In [ ]:
!unzip '/content/horse-or-human.zip'

In [ ]:
!unzip '/content/animal+utils.zip'

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets as dset
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import time
import matplotlib.pyplot as plt
import utils

### 2) Hyperparameter

In [ ]:
batch_size= 16 #64  #1
learning_rate = 0.0001
epoch = 20

n_node = 1024  # customized last layer 의 노드 수. 64, 128, 256, 512, 1024
dropratio = 0.5   # 얼마나 드랍시킬지 inverse keepratio 

imgsize = 256

## 2.Data Loader
### 트레이닝 데이터

In [ ]:
#img_dir='../../../images/painting_dataset/real_artwork_divided_shffl_4K/Train'
#img_dir="animal/train"
img_dir="train"
train_data=dset.ImageFolder(img_dir, transforms.Compose([
            transforms.CenterCrop(imgsize*2), #CenterCop(512)
            transforms.RandomCrop(imgsize), #RandomCrop
            transforms.RandomHorizontalFlip(), #RandomHorizontalFlip
            transforms.Resize(imgsize),
            transforms.ToTensor()    
            ]))
print(train_data.__len__())
train_batch=data.DataLoader(train_data, batch_size=batch_size,
                           shuffle=True,num_workers=2)

### 고정된 데이터 셋

In [ ]:
#2. Dev data
#img_dir='../../../images/painting_dataset/real_artwork_divided_shffl_4K/Valid'
# img_dir="animal/val"
img_dir="val"
dev_data=dset.ImageFolder(img_dir, transforms.Compose([
            transforms.CenterCrop(size=imgsize),
            transforms.Resize(imgsize),
            transforms.ToTensor()    
            ]))

dev_batch=data.DataLoader(dev_data, batch_size=batch_size,
                           shuffle=False,num_workers=2)

#3. Test data
# img_dir="animal/test"
img_dir="test"
test_data=dset.ImageFolder(img_dir, transforms.Compose([
            transforms.CenterCrop(size=imgsize),
            transforms.Resize(imgsize),
            transforms.ToTensor()    
            ]))

test_batch=data.DataLoader(test_data, batch_size=batch_size,
                           shuffle=False,num_workers=2)


nclass=len(train_data.classes)
print("# of classes: %d" %nclass)
print(train_data.classes)
print(train_data.class_to_idx)
print(train_data.__len__())

print("Training: %d, Dev: %d, Test: %d"
     %(train_data.__len__(), dev_data.__len__(), test_data.__len__()))

In [ ]:
print(train_data.classes)
print(dev_data.classes)
print(test_data.classes)

## 3.Model
### 1) Pretrained VGG Model

In [ ]:
vgg=models.vgg19(pretrained =True)

for name,module in vgg.named_children():
    print(name)
    
print(list(vgg.children())[0])
print(list(vgg.children())[-1])


In [ ]:
print(list(vgg.children())[0][0])

### 2)Customized Fully Model

In [ ]:
base_dim=64
fsize=int(imgsize/32)

class MyVGG(nn.Module):
    def __init__(self):
        super(MyVGG,self).__init__()
        #[0]: features(conv), [1]: classifier(fc)
        self.layer0=nn.Sequential(*list(vgg.children())[0])
        
        self.layer1=nn.Sequential(
            nn.Linear(8*base_dim*fsize*fsize,n_node),
            nn.BatchNorm1d(n_node),
            nn.ReLU(),
            nn.Dropout2d(dropratio),
        
            nn.Linear(n_node,n_node),
            nn.BatchNorm1d(n_node),
            nn.ReLU(),
            nn.Dropout2d(dropratio),
        
            nn.Linear(n_node,n_node),
            nn.BatchNorm1d(n_node),
            nn.ReLU(),
            nn.Dropout2d(dropratio),
            
            nn.Linear(n_node, nclass),        
            )
        
        for m in self.layer1.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal(m.weight.data)
                m.bias.data.fill_(0)
            if isinstance(m, nn.Linear):
                init.kaiming_normal(m.weight.data)
                m.bias.data.fill_(0)
                    
    def forward(self,x):
        out=self.layer0(x)
        out=out.view(out.size(0),-1)
        out=self.layer1(out)
        return out   

### 3) Model on GPU

In [ ]:
model=MyVGG().cuda()

for params in model.layer0.parameters():
    params.required_grad =False
    
for params in model.layer1.parameters():
    params.required_grad =True

In [ ]:
for name in model.children():
    print(name)

## 4. Optimizer & Loss

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.layer1.parameters(), lr=learning_rate)

## 5. Train

In [ ]:
import utils 
total_time=0
disp_step=10

to_train = True
if (to_train == False):
    # netname='./nets/catdog_vgg19_10.pkl'
    netname='./nets/horsehuman.pkl'
    model=torch.load(netname)
else:
    print("3 layer, n_node: %d, dropratio: %.2f" %(n_node,dropratio))
    model.eval()
    train_corr=utils.ComputeCorr(train_batch, model)
    dev_corr=utils.ComputeCorr(dev_batch, model)
    test_corr=utils.ComputeCorr(test_batch, model)
    print("Correct of train: %.2f, dev: %.2f, test: %.2f"
         %(train_corr, dev_corr, test_corr))
    model.train()
    netname='./nets/horsehuman1'
    
    x_epoch=[]
    y_train_err=[]
    y_dev_err=[]
    y_test_err=[]
    
    x_epoch.append(0)
    y_train_err.append(100.0-train_corr)
    y_dev_err.append(100.0-dev_corr)
    y_test_err.append(100.0-test_corr)
    
    for i in range(epoch):
        start_time=time.time()
        print("%d.."%i),
        for img,label in train_batch:
            img=Variable(img).cuda()
            label=Variable(label).cuda()
            
            optimizer.zero_grad()
            output=model(img)
            loss=loss_func(output,label)
            loss.backward()
            optimizer.step()
            
        end_time=time.time()
        duration=end_time-start_time
        total_time += duration
        if (i % disp_step==0) or (i==epoch-1):
            torch.save(model, netname+'_%d.pkl'%i,)
            print("\n[%d/%d] loss: %.3f, " %(i, epoch, (loss.cpu()).data.numpy())),
            model.eval()
            train_corr = utils.ComputeCorr(train_batch, model)
            dev_corr = utils.ComputeCorr(dev_batch, model)
            test_corr = utils.ComputeCorr(test_batch, model)
            print("Correct of train: %.2f, dev: %.2f, test: %2f, "
                 %(train_corr, dev_corr, test_corr)),
            model.train()
            print("time: %.2f sec.." %(total_time))
            
            x_epoch.append(i+1)
            y_train_err.append(100.0-train_corr)
            y_dev_err.append(100.0-dev_corr)
            y_test_err.append(100.0-test_corr)
        print("Total time: %.2f sec" %total_time)

In [ ]:
if (to_train):
    plt.plot(x_epoch, y_train_err, color='black', label='train err', linestyle='--')
    plt.plot(x_epoch, y_dev_err, color='red', label='dev err')
    plt.plot(x_epoch, y_test_err, color='blue', label='test err')
    
    plt.xlabel('epoch')
    plt.ylabel('err')
    plt.title('epoch&err graph')
    plt.legend(loc="upper right")
    plt.show()

## 6.Evaluation for dev & test data

In [ ]:
model.eval()
utils.EvaluateClassifier(dev_batch, model, dev_data.classes, batch_size)

In [ ]:
model.eval()
_, _,_ = utils.EvaluateClassifier(test_batch, model, test_data.classes, batch_size)

In [ ]:
utils.VisTFPred(dev_batch, model, test_data.classes, batch_size, i_n=2)